# Mode Selection
Set finetuning = 1 if you want to finetune, or set finetuning = 0 if you want to just load the saved model to get inference.

In [1]:
finetuning = 1
inference = not finetuning

# Drive Mount

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

# Read Data

In [3]:
import pandas as pd
data = pd.read_csv('/kaggle/input/gb-all-necessary/train_test_senti_data.csv')
data

,male,female,_original_label,test
0,সাইন্সের ছেলেরা আসলেই একটু অহংকারী ও দাম্বিক হ...,সাইন্সের মেয়েরা আসলেই একটু অহংকারী ও দাম্বিক হ...,0.0,0
1,কামাল এর ছাত্র এটা? এই ছাগটাকে প্রায় নাটকেই নে...,মেহজাবিন এর ছাত্রী এটা? এই ছাগীটাকে প্রায় নাটক...,0.0,0
2,পোলা টা খালি কান্দে,মাইয়া টা খালি কান্দে,0.0,0
3,ছাত্র টা কে যতবার দেখলাম ঘুষি মারতে ইচ্ছে হইছে...,ছাত্রী টা কে যতবার দেখলাম ঘুষি মারতে ইচ্ছে হইছ...,0.0,0
4,"ছেলেরা এরকমই হয়। সে ম্যাথ বুঝাতে পারেনি, আর ছে...","মেয়েরা এরকমই হয়। সে ম্যাথ বুঝাতে পারেনি, আর মে...",0.0,0
...,...,...,...,...
4847,অপুরভ ভাই অসাম,রাইসা ভাবী অসাম,1.0,0
4848,সেই ছিল এটা শিহাব ভাই,সেই ছিল এটা নূরজাহান আফা,1.0,0
4849,সেই ছিল এটা শিহাব ভাই,সেই ছিল এটা সুস্মিতা বোন,1.0,0
4850,সেই ছিল এটা শিহাব ভাই,সেই ছিল এটা গুলশান ভাবী,1.0,0


# Train, Test, Validation Split
We are going to consider the data as test data if the prediction between original sentence and NER converted sentence become unequal.

In [4]:
temp = data[data['test']!= 1]
test = data[data['test']== 1]

In [5]:
len(temp), len(test)

(3882, 970)

In [6]:
t1 = temp[['male', '_original_label']]
t2 = temp[['female', '_original_label']]
t2.columns = ['male', '_original_label']
temp = pd.concat([t1, t2], axis = 0)
temp

,male,_original_label
0,সাইন্সের ছেলেরা আসলেই একটু অহংকারী ও দাম্বিক হ...,0.0
1,কামাল এর ছাত্র এটা? এই ছাগটাকে প্রায় নাটকেই নে...,0.0
2,পোলা টা খালি কান্দে,0.0
3,ছাত্র টা কে যতবার দেখলাম ঘুষি মারতে ইচ্ছে হইছে...,0.0
4,"ছেলেরা এরকমই হয়। সে ম্যাথ বুঝাতে পারেনি, আর ছে...",0.0
...,...,...
4847,রাইসা ভাবী অসাম,1.0
4848,সেই ছিল এটা নূরজাহান আফা,1.0
4849,সেই ছিল এটা সুস্মিতা বোন,1.0
4850,সেই ছিল এটা গুলশান ভাবী,1.0


In [7]:
validation = temp.groupby('_original_label', group_keys=False).apply(lambda x: x.sample(frac=0.2, random_state=101))
train = temp.drop(validation.index)

<ipython-input-7-cf4ecadbd9c2>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  validation = temp.groupby('_original_label', group_keys=False).apply(lambda x: x.sample(frac=0.2, random_state=101))


In [8]:
train

,male,_original_label
1,কামাল এর ছাত্র এটা? এই ছাগটাকে প্রায় নাটকেই নে...,0.0
3,ছাত্র টা কে যতবার দেখলাম ঘুষি মারতে ইচ্ছে হইছে...,0.0
4,"ছেলেরা এরকমই হয়। সে ম্যাথ বুঝাতে পারেনি, আর ছে...",0.0
5,ছাত্র হিসেবে এই বান্দর ছ্যামড়ার ন্যাকামি দেইখা...,0.0
8,ওড়না ব্যবহার করলে কি খতি হত? খারাপ হয়ে যাচছে ব...,0.0
...,...,...
4844,তিশা আপুর অভিনয় খুব ভাল লেগেছে...?,1.0
4847,রাইসা ভাবী অসাম,1.0
4849,সেই ছিল এটা সুস্মিতা বোন,1.0
4850,সেই ছিল এটা গুলশান ভাবী,1.0


In [9]:
validation

,male,_original_label
292,ফালতু নাটক। নিশো ভাই একটু ভালো নাটক করেন না।,0.0
762,"বালের নাটক জা মনচায় তাই করে,,, সালা রা",0.0
261,ওরে ব্বাস মা কি ডেঞ্জারাস,0.0
690,কোন সালা জে এনাটকের পরি চালক এসব নাটক দেকে যুব...,0.0
393,দিক্কার জানাই এই সব পরিচালিকাকে যারা টাকার জন্...,0.0
...,...,...
2532,লক্ষী ভাবী ধন্যবাদ ভাবী জান এতো সুন্দর ময় নাটক...,1.0
1208,অসংখ্য ধন্যবাদ অমি ভাই এতো সুন্দর নাটক উপহার দ...,1.0
1557,ভাল্লাগছে ভাই এক কথায় অসাধারণ,1.0
1280,"এক কথাই অসাধারন,,,,,অভিরাম ভালবাসা নিশো ভাই ও ...",1.0


In [10]:
test

,male,female,_original_label,test
10,ছাত্র পড়ানো কঠিন প্যারা,ছাত্রী পড়ানো কঠিন প্যারা,0.0,1
11,"ছেলেরা টেকায় না পড়লে কখনো ছেলেদের কাছে জায় না,...","মেয়েরা টেকায় না পড়লে কখনো মেয়েদের কাছে জায় না,...",0.0,1
13,পোলা খোর কখন আসবো,মাইয়া খোর কখন আসবো,0.0,1
14,পোলা খোর কখন আসবো,মেয়া খোর কখন আসবো,0.0,1
17,মোশাররফ করিম ভাইয়া দিন দিন এতো বাজে নাটক করে ন...,আয়েশা আপু দিন দিন এতো বাজে নাটক করে নিম্মমানের।,0.0,1
...,...,...,...,...
4821,শিহাব শাহীন ভাইকে অনেক ধন্যবাদ।,সাহানা বোনকে অনেক ধন্যবাদ।,1.0,1
4827,শিহাব শাহিন এবং অপূর্ব মানে অন্যে রকম। তার মাঝ...,শিহাব শাহিন এবং রাইসা মানে অন্যে রকম। তার মাঝে...,1.0,1
4836,সত্যিই সিহাব ভাইজান তুমার প্রতি দোয়া রইরো আমাদ...,সত্যিই সিমন্তী বুবুজান তুমার প্রতি দোয়া রইরো আ...,1.0,1
4845,অপুরভ ভাই অসাম,মুসফিকা আফা অসাম,1.0,1


# Load Pretrained Model

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transform
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForMaskedLM, BertConfig

def load_transformer_based_model(model_path):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForMaskedLM.from_pretrained(model_path ,output_hidden_states=True)
    return tokenizer, model
model_path = "ka05ar/banglabert-sentiment"
tokenizer, b_model = load_transformer_based_model(model_path)
print('Total Pretrained tokens: ',len(tokenizer.get_vocab()))
tokenizer.add_tokens(['<Name>' ,'<Gender>'])
b_model.resize_token_embeddings(len(tokenizer))
print('Total number of tokens after adding the new ones: ',len(tokenizer.get_vocab()))

tokenizer_config.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/528k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/999 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of ElectraForMaskedLM were not initialized from the model checkpoint at ka05ar/banglabert-sentiment and are newly initialized: ['generator_lm_head.bias', 'generator_predictions.LayerNorm.bias', 'generator_predictions.LayerNorm.weight', 'generator_predictions.dense.bias', 'generator_predictions.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Total Pretrained tokens:  32000


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Total number of tokens after adding the new ones:  32002


# Dataset Preparation

In [33]:
import pandas as pd
import numpy as np
import transformers
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from tqdm import tqdm



class BertDataset(Dataset):
    def __init__(self, data, tokenizer, train , gender, max_length):
        super(BertDataset, self).__init__()

        self.train_csv= data
        self.tokenizer=tokenizer
        self.target=self.train_csv.iloc[:,1]
        self.max_length=max_length
        self.train = train
        self.gender = gender

    def __len__(self):
        return len(self.train_csv)

    def __getitem__(self, index):

        if self.train == 1:
            text1 = self.train_csv.iloc[index,0] #0th column all data
        else:
            if self.gender == 'male':
                text1 = self.train_csv.iloc[index,0] #male
            else:
                text1 = self.train_csv.iloc[index,1] #female
                
                

        inputs = self.tokenizer.encode_plus(
            text1 ,
            None,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            max_length=self.max_length,
            truncation = True
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]
        
        try:
            if self.train == 1:
              return torch.tensor(ids, dtype=torch.long), torch.tensor(token_type_ids, dtype=torch.long), torch.tensor(mask, dtype=torch.long), torch.tensor(self.train_csv.iloc[index, 1], dtype=torch.long)
            else:
              return torch.tensor(ids, dtype=torch.long), torch.tensor(token_type_ids, dtype=torch.long), torch.tensor(mask, dtype=torch.long), torch.tensor(self.train_csv.iloc[index, 2], dtype=torch.long)
                
        except:
          print(index, self.train_csv[index])

dataset_train= BertDataset(train, tokenizer, train = 1 , gender = None, max_length = 512)
dataset_val= BertDataset( validation, tokenizer, train = 1 , gender = None, max_length = 512)
dataset_test_male = BertDataset(test, tokenizer, train = 0 , gender = 'male', max_length = 512)
dataset_test_female = BertDataset(test, tokenizer, train = 0 , gender = 'female', max_length = 512)

train_loader=DataLoader(dataset=dataset_train,batch_size=16, shuffle=True)
validation_loader=DataLoader(dataset=dataset_val,batch_size=16, shuffle=False)
test_loader_male=DataLoader(dataset=dataset_test_male,batch_size=16, shuffle=False)
test_loader_female=DataLoader(dataset=dataset_test_female,batch_size=16, shuffle=False)

In [13]:
# id,am, tid, l = next(iter(train_loader))
# x = b_model(id, am,tid)
# x.hidden_states[0].shape

# Model

In [14]:

class BERT(nn.Module):
    def __init__(self, model):
        super(BERT, self).__init__()
        self.bert_model = model
        self.dropout = nn.Dropout(0.2)
        self.out = nn.Linear(768, 1)

    def forward(self,ids,mask,token_type_ids):
        o2= self.bert_model(ids,mask,token_type_ids).hidden_states[0]
        o2= o2.max(dim=1).values
        o2 = self.dropout(o2)
        # print('------->>>>>>>>>',o2.values.shape)
        out= self.out(o2)

        return out, o2

masked_model=BERT(b_model)
# loss_fn = nn.BCEWithLogitsLoss()
# #Initialize Optimizer
# optimizer= optim.Adam(model.parameters(),lr= 0.0001)

In [15]:
# id,am, tid, l = next(iter(train_loader))
# x,_ = masked_model(id, am,tid)
# x.shape

# Training Loop Decorator

In [16]:
from collections import OrderedDict
from collections import namedtuple
from itertools import product
import json
from torch.utils.tensorboard import SummaryWriter
import time
import torchvision
import pandas as pd
import torch
from IPython.display import display
from IPython.display import clear_output

class RunBuilder():
    @staticmethod
    def get_runs(params):
        Run = namedtuple('Run', params.keys())


        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))


        return runs


class RunManager():
    def __init__(self):
        self.epoch_count = 0
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_start_time = None

        self.epoch_valid_loss = 0.0
        self.epoch_num_valid_correct = 0

        self.run_params = None
        self.run_count = 0
        self.run_data = []
        self.run_start_time = None

        self.network = None
        self.loader = None
        self.validation_loader = None
        self.tb = None

    def begin_run(self, run, network, loader, validation_loader):
        self.run_start_time = time.time()

        self.run_params = run
        self.run_count += 1

        self.network = network
        self.loader = loader
        self.validation_loader = validation_loader
        self.tb = SummaryWriter(comment=f'-{run}')



    def end_run(self):
        self.tb.close()
        self.epoch_count = 0

    def begin_epoch(self):
        self.epoch_start_time = time.time()
        self.epoch_count += 1
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_valid_loss = 0
        self.epoch_num_valid_correct = 0

    def end_epoch(self):
        epoch_duration = time.time() - self.epoch_start_time
        run_duration = time.time() - self.run_start_time

        loss = self.epoch_loss/len(self.loader.dataset)
        accuracy = self.epoch_num_correct/len(self.loader.dataset)
        val_loss = self.epoch_valid_loss/len(self.validation_loader.dataset)
        val_accuracy = self.epoch_num_valid_correct/len(self.validation_loader.dataset)

        self.tb.add_scalar('Loss', loss, self.epoch_count)
        self.tb.add_scalar('Accuracy', accuracy, self.epoch_count)
        self.tb.add_scalar('Validation Loss', val_loss, self.epoch_count)
        self.tb.add_scalar('Validation Accuracy', val_accuracy, self.epoch_count)

        for name, param in self.network.named_parameters():
            self.tb.add_histogram(name, param, self.epoch_count)

        results = OrderedDict()
        results["run"] = self.run_count
        results["epoch"] = self.epoch_count
        results["loss"] = loss
        results["accuracy"] = accuracy
        results["validation loss"] = val_loss
        results["validation accuracy"] = val_accuracy
        results["epoch duration"] = epoch_duration
        results["run duration"] = run_duration
        for k,v in self.run_params._asdict().items(): results[k] = v
        self.run_data.append(results)
        df = pd.DataFrame.from_dict(self.run_data, orient='columns')

        clear_output(wait=True)
        display(df)

    def track_loss(self, loss):
        self.epoch_loss += loss.item()#*self.loader.batch_size

    def track_num_correct(self, preds, labels):
        self.epoch_num_correct += self._get_num_correct(preds, labels)

    def track_validation_loss(self, loss):
        self.epoch_valid_loss += loss.item()#*self.loader.batch_size

    def track_num_validation_correct(self, preds, labels):
        self.epoch_num_valid_correct += self._get_num_correct(preds, labels)

    @torch.no_grad()
    def _get_num_correct(self, preds, labels):
        return torch.where(preds >= 0, 1, 0).eq(labels).sum().item()

    def save(self, fileName):
        pd.DataFrame.from_dict(
            self.run_data,
            orient='columns').to_csv(rf'{fileName}.csv')


# Training

In [17]:
SAVE_MODEL = True
from tqdm import tqdm
import torch
import torch.optim as optim
import torch.nn.functional as F
from collections import OrderedDict
from datetime import datetime
from transformers import get_linear_schedule_with_warmup, AdamW
def my_trainer(train_loader, validation_loader, network, LR = 1e-5 ,epochs = 5, BATCH_SIZE = 16,  fresh_training = True, validation = True):
    params = OrderedDict(
        lr = [LR],
        batch_size =[BATCH_SIZE],
        device = ['cuda' if torch.cuda.is_available() else 'cpu'],
    )

    m = RunManager()
    for run in RunBuilder.get_runs(params):
        device = torch.device(run.device)
        network = network.to(device)
        # optimizer = optim.Adam(network.parameters(), lr=run.lr)

        loss_fn = nn.BCEWithLogitsLoss()
        total_steps = len(train_loader) * epochs
        optimizer = optim.Adam(network.parameters(), lr= run.lr)

        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)
        if not fresh_training:
            load_model(network, optimizer, PATH = 'foa_senti.pth')
        m.begin_run(run, network, train_loader, validation_loader)
        for epoch in range(epochs):
            network.train(True)
            print(f'Epoch {epoch + 1} : ', end = '\t')
            m.begin_epoch()
            for batch in tqdm(train_loader):
                input_ids, token_type_ids, attention_masks, labels = batch
                input_ids = input_ids.to(device)
                token_type_ids = token_type_ids.to(device)
                attention_masks = attention_masks.to(device)
                labels = labels.type(torch.LongTensor)
                labels = labels.to(device)

                preds,hidden_states = network(input_ids, attention_masks,token_type_ids)
                labels = labels.type_as(preds).view(-1, 1)
                # print(labels.shape)
                # print('----------')
                # print(preds.shape)


                loss = loss_fn(preds, labels)
                # print(preds.shape, labels.shape)
                # print(preds, labels)


                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(network.parameters(), 1.0)
                optimizer.step()
                scheduler.step()


                m.track_loss(loss)
                m.track_num_correct(preds, labels)


            network.eval()
            for batch in tqdm(validation_loader):
                input_ids, token_type_ids, attention_masks, labels = batch
                input_ids = input_ids.to(device)
                token_type_ids = token_type_ids.to(device)
                attention_masks = attention_masks.to(device)
                labels = labels.type(torch.LongTensor)
                labels = labels.to(device)
                with torch.no_grad():
                    preds,hidden_states = network(input_ids, attention_masks,token_type_ids)
                    labels = labels.type_as(preds).view(-1, 1)
                    loss = loss_fn(preds, labels)
                    m.track_validation_loss(loss)
                    m.track_num_validation_correct(preds, labels)
                # break #not doing validation here




            m.end_epoch()
            network.train(False)
        m.end_run()
    if SAVE_MODEL == True:
        save_model(network, optimizer)

    s = datetime.now().strftime("%d-%m-%Y %I:%M:%p").replace(" ", "_").replace(":", "_")
    PATH  = rf'results_{s}'
    m.save(PATH)
    return


def save_model(network, optimizer):
    s = datetime.now().strftime("%d-%m-%Y %I:%M:%p").replace(" ", "_").replace(":", "_")
    PATH  = rf'foa_senti.pth'
    torch.save({
    'model_state_dict': network.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    }, PATH)
    print(f'Saved model and optimizer at {PATH}')
    return

def load_model(network, optimizer, PATH ):
    checkpoint = torch.load(PATH)
    network.load_state_dict(checkpoint['model_state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print(rf'Loaded model and optimizer from {PATH}')
    else:
        print(rf'Loaded model from {PATH}')

    return

def get_parameter_info(network, details = False):
    print('Total parameters:' ,(sum(p.numel() for p in network.parameters())))
    print('Total trainable parameters:' ,(sum(p.numel() for p in network.parameters() if p.requires_grad)))
    if details:
        print('---------------------------------\nDetailed Parameter Info: ')

        for name, parameter in network.named_parameters():
            print('name, parameter.requires_grad, parameter.is_cuda, parameter.size(): ', name, parameter.requires_grad, parameter.is_cuda, parameter.size())
            print('---------------------------------')
    return


if finetuning:
  my_trainer(train_loader, validation_loader, masked_model, 0.0001, epochs = 15, BATCH_SIZE = 16, fresh_training = True)

,run,epoch,loss,accuracy,validation loss,validation accuracy,epoch duration,run duration,lr,batch_size,device
0,1,1,0.029705,0.798746,0.026850,0.797167,148.576178,148.578425,0.0001,16,cuda
1,1,2,0.022507,0.816343,0.020862,0.801030,148.253354,299.908192,0.0001,16,cuda
2,1,3,0.015523,0.900890,0.015324,0.888603,148.191148,450.657367,0.0001,16,cuda
3,1,4,0.009852,0.960963,0.011658,0.932389,148.181154,601.355635,0.0001,16,cuda
4,1,5,0.006748,0.977751,0.010023,0.935608,148.243376,752.131089,0.0001,16,cuda
5,1,6,0.004913,0.983212,0.008626,0.947199,148.183565,902.805677,0.0001,16,cuda
6,1,7,0.003796,0.988066,0.008093,0.941404,148.210057,1053.700134,0.0001,16,cuda
7,1,8,0.003020,0.990089,0.007788,0.943335,148.261483,1204.543490,0.0001,16,cuda
8,1,9,0.002573,0.992921,0.007438,0.943335,148.268803,1355.419539,0.0001,16,cuda
9,1,10,0.002245,0.992516,0.007038,0.949131,148.178525,1506.092304,0.0001,16,cuda


Saved model and optimizer at foa_senti.pth


# Load Model

In [17]:
#Load Model
from tqdm import tqdm
import torch
import torch.optim as optim
import torch.nn.functional as F
from collections import OrderedDict
from datetime import datetime
from transformers import get_linear_schedule_with_warmup, AdamW

def load_model(network, optimizer, PATH ):
    checkpoint = torch.load(PATH, map_location=torch.device('cpu'))
    network.load_state_dict(checkpoint['model_state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print(rf'Loaded model and optimizer from {PATH}')
    else:
        print(rf'Loaded model from {PATH}')

    return


if inference:
  load_model(masked_model, None, PATH = '/kaggle/input/foa_senti/pytorch/default/1/foa_senti.pth')

<ipython-input-17-cca5971061c9>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(PATH, map_location=torch.device('cpu'))


Loaded model from /content/drive/MyDrive/#Research/# GB/saved_model/baseline_senti.pth


# Inference

In [35]:
def get_prediction(data_loader, network):
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  device = torch.device(device)
  network = network.to(device)
  network.eval()
  l = []
  emb = []
  for batch in tqdm(data_loader):
    input_ids, token_type_ids, attention_masks, labels = batch
    input_ids = input_ids.to(device)
    token_type_ids = token_type_ids.to(device)
    attention_masks = attention_masks.to(device)
    labels = labels.type(torch.LongTensor)
    labels = labels.to(device)
    with torch.no_grad():
        preds, hidden_states = network(input_ids, attention_masks,token_type_ids)
        labels = labels.type_as(preds).view(-1, 1)
        preds =  torch.where(preds >= 0, 1, 0)
        l.append(preds)
        emb.append(hidden_states)
  return l, torch.vstack(emb).cpu().numpy()

import warnings 
warnings.filterwarnings('ignore') 

p, emb_male = get_prediction(test_loader_male, masked_model)
test['pred_male_fod'] = [x[0] for pred in p for x in pred.tolist()]
p, emb_male = get_prediction(test_loader_female, masked_model)
test['pred_female_fod'] = [x[0] for pred in p for x in pred.tolist()]
x = sum(test['pred_male_fod']!=test['_original_label'])
print('For Male:\nTotal test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1- (x/len(test))) 
x = sum(test['pred_female_fod']!=test['_original_label'])
print('For Female:\nTotal test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1- (x/len(test)))
print('Male Female Mismatch: ', sum(test['pred_male_fod']!=test['pred_female_fod']))

100%|██████████| 61/61 [00:20<00:00,  3.01it/s]


For Male:
Total test data:  970 Total mismatch:  45 Accuracy:  0.9536082474226804
For Female:
Total test data:  970 Total mismatch:  52 Accuracy:  0.9463917525773196
Male Female Mismatch:  21


# FOA Result

In [36]:
import numpy as np
np.random.seed(0)

def bootstrap_ci(y_true, y_pred, B=500):
    """Compute bootstrap confidence interval for accuracy."""
    N = len(y_true)
    original_acc = np.mean(y_pred == y_true)

    bootstrap_accs = []
    
    for _ in range(B):
        indices = np.random.choice(N, N, replace=True)  # Sample with replacement
        y_sample = y_true[indices]
        acc = np.mean(y_pred[indices] == y_sample)
        bootstrap_accs.append(acc)
    # Compute 95% confidence interval
    lower, upper = np.percentile(bootstrap_accs, [2.5, 97.5])
    
    return original_acc, (lower, upper), np.mean(bootstrap_accs)

y_true = test['_original_label'].values  # Ground truth labels
y_pred = test['pred_male_fod'].values
acc, ci, ma = bootstrap_ci(y_true, y_pred)
print('Male: ')
print(f"Accuracy: {acc:.4f}")
print(f"95% Confidence Interval: {ci}")
print(f"mean accuracy: {ma}")

y_true = test['_original_label'].values
y_pred = test['pred_female_fod'].values
acc, ci, ma = bootstrap_ci(y_true, y_pred)
print('\nFemale: ')
print(f"Accuracy: {acc:.4f}")
print(f"95% Confidence Interval: {ci}")
print(f"mean accuracy: {ma}")
test.to_csv('2,3.revision_foa_senti_result.csv', index = False)

Male: 
Accuracy: 0.9536
95% Confidence Interval: (0.9412371134020618, 0.965979381443299)
mean accuracy: 0.9539154639175258

Female: 
Accuracy: 0.9464
95% Confidence Interval: (0.9309278350515464, 0.9597938144329897)
mean accuracy: 0.9462103092783505


In [ ]:
# Statistical Parity Difference (SPD) and Equal Opportunity Difference (EOD) Calculation
import numpy as np
import pandas as pd
np.random.seed(0)

test = pd.read_csv('revision_foa_senti_result.csv')


# Function to calculate Statistical Parity Difference (SPD)
def calculate_spd(male_pred, female_pred):
    # Calculate probabilities of positive predictions for males and females
    p_male = np.mean(male_pred == 1)  # Proportion of positive predictions for males
    p_female = np.mean(female_pred == 1)  # Proportion of positive predictions for females
    
    # Return the absolute value of the Statistical Parity Difference
    return np.abs(p_male - p_female)

# Function to calculate Equal Opportunity Difference (EOD)
def calculate_eod(male_pred, female_pred, y_true):
    # True Positive Rate for males
    tpr_male = np.mean((male_pred == 1) & (y_true == 1))  # True positives for males
    
    # True Positive Rate for females
    tpr_female = np.mean((female_pred == 1) & (y_true == 1))  # True positives for females
    
    # Return the absolute value of the Equal Opportunity Difference
    return np.abs(tpr_male - tpr_female)

# Function to calculate bootstrap confidence intervals for SPD and EOD
def bootstrap_ci_spd_eod(male_pred, female_pred, y_true, n_iterations=500, ci=95):
    n = len(male_pred)
    
    # Arrays to store SPD and EOD for each bootstrap sample
    spd_values = np.zeros(n_iterations)
    eod_values = np.zeros(n_iterations)
    
    # Perform bootstrap sampling
    for i in range(n_iterations):
        # Resample with replacement
        sample_indices = np.random.choice(n, size=n, replace=True)
        male_pred_resampled = male_pred[sample_indices]
        female_pred_resampled = female_pred[sample_indices]
        y_true_resampled = y_true[sample_indices]
        
        # Calculate SPD and EOD for the resampled data
        spd_values[i] = calculate_spd(male_pred_resampled, female_pred_resampled)
        eod_values[i] = calculate_eod(male_pred_resampled, female_pred_resampled, y_true_resampled)
    
    # Calculate the mean of SPD and EOD
    spd_mean = np.mean(spd_values)
    eod_mean = np.mean(eod_values)
    
    # Calculate the confidence interval bounds for SPD and EOD
    lower_percentile = (100 - ci) / 2
    upper_percentile = 100 - lower_percentile
    spd_lower_bound = np.percentile(spd_values, lower_percentile)
    spd_upper_bound = np.percentile(spd_values, upper_percentile)
    eod_lower_bound = np.percentile(eod_values, lower_percentile)
    eod_upper_bound = np.percentile(eod_values, upper_percentile)
    
    return (np.abs(spd_mean), np.abs(spd_lower_bound), np.abs(spd_upper_bound)), (np.abs(eod_mean), np.abs(eod_lower_bound), np.abs(eod_upper_bound))


y_true = test._original_label.values
male_pred = test.pred_male_fod.values
female_pred = test.pred_female_fod.values

# Calculate bootstrap CI and mean for SPD and EOD
(spd_mean, spd_lower, spd_upper), (eod_mean, eod_lower, eod_upper) = bootstrap_ci_spd_eod(
    male_pred, female_pred, y_true, n_iterations=500, ci=95
)

print(f"SPD 95% Confidence Interval: [{spd_lower:.3f}, {spd_upper:.3f}]\tSPD Mean: {spd_mean:.3f}")
print('---------------------------------------------------------------------------------------------')
print(f"EOD 95% Confidence Interval: [{eod_lower:.3f}, {eod_upper:.3f}]\tEOD Mean: {eod_mean:.3f}")

SPD 95% Confidence Interval: [0.000, 0.012]	SPD Mean: 0.004
---------------------------------------------------------------------------------------------
EOD 95% Confidence Interval: [0.000, 0.008]	EOD Mean: 0.003


# Mismatched Samples

In [37]:
test[test['pred_male_fod']!=test['pred_female_fod']]

,male,female,_original_label,test,pred_male_fod,pred_female_fod
264,সালা আসলেই ফাজিল!,সালী আসলেই ফাজিল!,0.0,1,1,0
279,পার্কে এক ভাইয়া দাঁড়িয়ে দাড়িয়ে কিস খাইতেছিলো।ব...,পার্কে এক আপু দাঁড়িয়ে দাড়িয়ে কিস খাইতেছিলো।বলল...,0.0,1,0,1
314,ছেলেদের বাজে ছেলে অনেক পছন্দ,মেয়েদের বাজে মেয়ে অনেক পছন্দ,0.0,1,0,1
319,কামাল ছেলেদের গায়ে বেশি বেশি ওড়তে ছায় পালতু ছ...,কালীতারা মেয়েদের গায়ে বেশি বেশি ওড়তে ছায় পালতু...,0.0,1,0,1
422,শালার নিশো ভাইয়ের অভিনয় মানে জটিলআর আতিকুরের খ...,শালীর সাহানা ভাবীর অভিনয় মানে জটিলআর বিনিতার খ...,0.0,1,1,0
430,"এরকমের ছেলেদের স্টাইলিশ না মানুষ ""মেজাই "" বলে।...","এরকমের মেয়েদের স্টাইলিশ না মানুষ ""মেজাই "" বলে...",0.0,1,0,1
431,ফিনিশিং টা ভালো লাগেনি। এধরনের ছেলেদের রিয়েল ল...,ফিনিশিং টা ভালো লাগেনি। এধরনের মেয়েদের রিয়েল ল...,0.0,1,0,1
436,"নিশো পাগলা টা ফাটাইয়া দিছে,,,","নুজহাত পাগলি টা ফাটাইয়া দিছে,,,",0.0,1,0,1
451,আহারে বেচারা শামীম চড়ের ওপরে আছে,আহারে বেচারি আদৃতা চড়ের ওপরে আছে,0.0,1,0,1
898,আমার শাওয়ার নাটক! শাওয়ার স্ক্রিপ্ট! শাওয়ার পরি...,আমার শাওয়ার নাটক! শাওয়ার স্ক্রিপ্ট! শাওয়ার পরি...,0.0,1,1,0


In [20]:
# After Finetuning
x = sum(test['original_lebel'] != test['pred'])
print('Total test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)) )

Total test data:  218 Total mismatch:  26 Accuracy:  0.8807339449541285


In [ ]:
# After Finetuning
x = sum(test['original_lebel'] != test['pred'])
print('Total test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)) )

Total test data:  218 Total mismatch:  26 Accuracy:  0.8807339449541285


In [ ]:
# Before fine tuning
sum(data['Predicted Value for Original Sentence'] != data['Predicted Value for ner converted'])

218


# Prediction of NER Converted text data

In [21]:
import pandas as pd
import numpy as np
import transformers
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from tqdm import tqdm


class BertDatasetNER(Dataset):
    def __init__(self, data, tokenizer,max_length):
        super(BertDatasetNER, self).__init__()

        self.train_csv= data
        self.tokenizer=tokenizer
        self.target=self.train_csv.iloc[:,1]
        self.max_length=max_length

    def __len__(self):
        return len(self.train_csv)

    def __getitem__(self, index):

        text1 = self.train_csv.iloc[index,1] #1 column for NER Converted

        inputs = self.tokenizer.encode_plus(
            text1 ,
            None,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            max_length=self.max_length,
            truncation = True
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]
        try:
          return torch.tensor(ids, dtype=torch.long), torch.tensor(token_type_ids, dtype=torch.long), torch.tensor(mask, dtype=torch.long), torch.tensor(self.train_csv.iloc[index, 3], dtype=torch.long)
        except:
          print(index, self.train_csv[index])



dataset_test= BertDatasetNER(test, tokenizer, max_length=200)
test_loader=DataLoader(dataset=dataset_test,batch_size=16, shuffle = False)
p, emb = get_prediction(test_loader, masked_model)
test['pred_ner'] = [x[0] for pred in p for x in pred.tolist()]
x = sum(test['original_lebel'] != test['pred_ner'])
print('Total test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)), 'Bias: ', sum(test['pred'] != test['pred_ner']) )

100%|██████████| 14/14 [00:02<00:00,  4.99it/s]

Total test data:  218 Total mismatch:  35 Accuracy:  0.8394495412844036 Bias:  17


In [22]:
print('Total test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)), 'Bias: ', sum(test['pred'] != test['pred_ner']) )

Total test data:  218 Total mismatch:  35 Accuracy:  0.8394495412844036 Bias:  17


In [ ]:
print('Total test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)), 'Bias: ', sum(test['pred'] != test['pred_ner']) )

Total test data:  218 Total mismatch:  36 Accuracy:  0.8348623853211009 Bias:  18


In [ ]:
test.to_csv('/content/drive/MyDrive/#Research/# GB/baseline_senti_reSult.csv', index = False)